<a href="https://colab.research.google.com/github/WSD2019-teamD/data_reduction/blob/master/colab/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#colabの日本語設定
インストール してランタイムをリスタートする

In [0]:
# from pathlib import Path

# for fontlist in Path("/root/.cache/matplotlib/").glob("*.json"):
#     with open(fontlist) as file:
#         for line in file:
#             if '"name"' in line:
#                 print(line.rstrip())

In [2]:
!apt-get -y install fonts-ipafont-gothic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-ipafont-gothic is already the newest version (00303-18ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [3]:
import matplotlib
matplotlib.get_cachedir()

'/root/.cache/matplotlib'

In [4]:
rm /root/.cache/matplotlib/fontList.json

rm: cannot remove '/root/.cache/matplotlib/fontList.json': No such file or directory


一度ランタイムをリセットする

#colabの設定

In [0]:
# from google.colab import auth
# auth.authenticate_user()

In [6]:
pip install gensim --upgrade

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.1)


In [7]:
pip list | grep gensim

gensim                   3.8.1      


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 準備
データをCSVから読み込む

In [0]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import re
# import MeCab
import random
from gensim import corpora, models
import pandas as pd
from tqdm import tqdm

import pickle
import gensim

In [10]:
import os
os.chdir('/content/drive/My Drive/Colab_Notebooks/data/article')
print(os.getcwd())

/content/drive/My Drive/Colab_Notebooks/data/article


# 辞書を作成

In [35]:
dir_path = os.curdir
print(dir_path)

files = [path for path in os.listdir(dir_path)]
print(files)

.
['2019-01-31_00:00:00-content.csv', '2018-12-31_00:00:00-content.csv', '2018-10-31_00:00:00-content.csv', '2018-09-30_00:00:00-content.csv', '2018-08-31_00:00:00-content.csv', '2018-07-31_00:00:00-content.csv', '2018-06-30_00:00:00-content.csv', '2018-05-31_00:00:00-content.csv', '2018-04-30_00:00:00-content.csv', '2018-03-31_00:00:00-content.csv', '2018-02-28_00:00:00-content.csv', 'model.tfidf', 'wordcloud.png']


In [0]:
# place = './drive/My\ Drive/Colab_Notebooks/data/article/'
df = pd.DataFrame()
for file in files[:1]:
    df_buf =  pd.read_csv(file)
    df = pd.concat([df, df_buf], axis = 0,  ignore_index=True)

In [37]:
df.head(1)

,Unnamed: 0,id,article_id,title,created_at,likes_count,tags_str,url,sharing
0,0,30,b9c5eb649ada9224e928,CloudFunctionsからCloudStorageへ画像をアップロードする,2019-01-31 00:01:26,8,"CloudStorage,Firebase,FirebaseCloudFunctions",https://qiita.com/shim0mura/items/b9c5eb649ada...,"やる, てる, の, cloudfunctions, ある, ファイル, 画像, 読み込める..."


In [14]:
share_list = []
for sh in df["sharing"]:
#     print(share)
    if type(sh) is str:
        share_list.append(sh.split(", "))
    else:
        share_list.append([])

len(share_list)

8225

In [0]:
d = corpora.Dictionary(share_list)

In [0]:
from smart_open import open
logging.getLogger('smart_open').setLevel(logging.ERROR)
d.filter_extremes(no_below=5, no_above=0.2)
# nob_below以下の個数の単語を無視
# no_aboveの割合以上に出てくる単語を無視
d.compactify()
# idを振り直してコンパクトにする。

# errorになる　ローカル環境（colabじゃないならOK）
# logging.basicConfig(format='%(message)s', level=logging.INFO)
# d.save_as_text("../dictionary/dictionary.txt")

In [17]:
print(len(d))
print(d[1289])
print("python id:",d.token2id["python"])
print("C++ id:",d.token2id["C++"])
print(d.doc2bow(["探索","ノード"]))

13930
lts
python id: 1362
C++ id: 3805
[(3347, 1), (3965, 1)]


In [18]:
[(d[t[0]],t[1])for t in sorted(d.dfs.items(),key=lambda x:x[1],reverse=True)][:10]

[('データ', 1630),
 ('簡単', 1622),
 ('問題', 1594),
 ('取得', 1586),
 ('コマンド', 1573),
 ('情報', 1563),
 ('いい', 1561),
 ('機能', 1480),
 ('エラー', 1478),
 ('実装', 1477)]

## コーパスの作成
辞書を作成したら、doc2bow() という関数を用いて、トークン化された文書をベクトルに変換します。

doc2bow() は、単純にそれぞれの単語の出現回数を計算した後、単語はIDに変換し、その結果を疎ベクトルとして返します。

In [19]:
os.path.exists("../corpus/corpus.pkl")

True

In [0]:
corpus = [d.doc2bow(w) for w in share_list]

# save corpus
with open("../corpus/corpus.pkl",'wb') as f:
    pickle.dump(corpus,f)

# corpus[:3][:3][:3]

In [0]:
## 読み取りでも良い
def get_corpus():
  if(os.path.exists("../corpus/corpus.pkl")):
    f = open("../corpus/corpus.pkl",'rb')
    return pickle.load(f)
  else:
    print("error")

# corpus = get_corpus()


### option
TFIDFコーパスに変更する

BoWコーパスとの違いは重み付けをしているかどうか

\begin{eqnarray}
  tf= \frac{文書Aにおける単語Xの出現頻度}{文書Aにおける全単語の出現頻度の和} \\
    df=log(全文書数単語Xを含む文書数) \\
    tfidf=tf∗idf
\end{eqnarray}

tfidf=(単語の出現頻度)∗(各単語のレア度)

In [0]:
# tfidf
tfidf = gensim.models.TfidfModel(corpus)
tfidf.save('model.tfidf')

# make corpus_tfidf
corpus_tfidf = tfidf[corpus]

# save corpus_tfidf
with open("../corpus/corpus_tfidf.pkl",'wb') as f:
    pickle.dump(corpus_tfidf,f)


# LDAの学習

In [0]:
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [24]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=50, passes=10)

using symmetric alpha at 0.02
using symmetric eta at 0.02
using serial LDA version on this node
running online (multi-pass) LDA training, 50 topics, 10 passes over the supplied corpus of 8225 documents, updating model once every 2000 documents, evaluating perplexity every 8225 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/8225
merging changes from 2000 documents into a model of 8225 documents
topic #12 (0.020): 0.018*"データ" + 0.008*"値" + 0.007*"関数" + 0.006*"コマンド" + 0.005*"()" + 0.005*"文字列" + 0.005*"クラス" + 0.005*"プロパティ" + 0.004*"参照" + 0.003*"クリック"
topic #22 (0.020): 0.012*"データ" + 0.007*"生成" + 0.006*"計算" + 0.004*"取得" + 0.004*"値" + 0.004*"簡単" + 0.004*"メソッド" + 0.004*"関数" + 0.004*"入力" + 0.004*"実装"
topic #21 (0.020): 0.007*"コマンド" + 0.006*"テスト" + 0.005*"接続" + 0.005*"エラー" + 0.005*"boldsymbol" + 0.004*"学習" + 0.004*"情報" + 0.004*"mathrm" + 0.004*"データ" + 0.003*"入力"
topic #23 (0.020): 0.014*"データ" + 0.006*"取得" + 0.005*"やすい" + 0.005*"モデル" + 0.005

In [25]:
def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))
for t in range(10):
    print("Topic # ",t)
    get_topic_words(t)
    print("\n")

Topic #  0
動画: 0.09212564677000046
ANDROID: 0.061655495315790176
セッション: 0.047589462250471115
アプリ: 0.03278973698616028
資料: 0.02420412190258503
iOS: 0.020052241161465645
カテゴリ: 0.019514475017786026
ポーズ: 0.018034018576145172
SSD: 0.01675269939005375
スピーカー: 0.015730513259768486


Topic #  1
Qiita: 0.05703644081950188
ver: 0.04497532173991203
カード: 0.04494384303689003
追記: 0.03487766534090042
よい: 0.01949552819132805
プログラム: 0.018704557791352272
文書: 0.015516949817538261
pdf: 0.015174313448369503
NRI: 0.014854246750473976
PO: 0.014583880081772804


Topic #  2
bit: 0.03932519629597664
USB: 0.03129149600863457
電源: 0.017283841967582703
スイッチ: 0.01637188158929348
プログラム: 0.01511848159134388
ケーブル: 0.014286089688539505
接続: 0.014103122986853123
出力: 0.013023938983678818
写真: 0.012751543894410133
OS: 0.011028770357370377


Topic #  3
データ: 0.034427985548973083
サイズ: 0.022061266005039215
XY: 0.020817162469029427
size: 0.019025659188628197
次元: 0.014177992939949036
シミュレーション: 0.012545453384518623
クラスタリング: 0.012482

In [0]:
def basic_info(num):
  if(num > len(df)):
    print("error")
    return
  print(df['url'][num])
  print(df['title'][num])
  print('like count: ' + str(df['likes_count'][num]))

In [0]:
def document_corpus(num):
  print('文書のトピックの関連')
  topics = sorted(lda.get_document_topics(corpus[0]), key=lambda t:t[1], reverse=True)
  basic_info(num)
  print('-' * 20) 
  for t in topics[:5]:
    print("Topic # ",t[0])
    get_topic_words(t[0])
    print("\n")
    # print("{}: {}".format(t[0], t[1]))


In [28]:
document_corpus(20)

文書のトピックの関連
https://qiita.com/kirurobo/items/82dd484ad6374e725a43
Unity上での自ウィンドウハンドル取得方法5種
like count: 1
--------------------
Topic #  12
データ: 0.057671546936035156
出力: 0.03654100000858307
文字列: 0.02407602034509182
変換: 0.017677754163742065
取得: 0.016649194061756134
CSV: 0.014154926873743534
形式: 0.01379439327865839
読み込む: 0.010133190080523491
番号: 0.009533041156828403
入力: 0.009136069566011429


Topic #  34
取得: 0.02737988345324993
情報: 0.023270325735211372
URL: 0.020943526178598404
登録: 0.020065490156412125
アクセス: 0.01697816140949726
ページ: 0.015398483723402023
ログイン: 0.01418359111994505
id: 0.013560472056269646
サイト: 0.012558979913592339
API: 0.012379884719848633


Topic #  33
API: 0.04299360513687134
リクエスト: 0.03241325169801712
サーバー: 0.020502695813775063
取得: 0.017708633095026016
送信: 0.016485661268234253
client: 0.016410598531365395
JSON: 0.016217926517128944
データ: 0.015857679769396782
HTTP: 0.013819776475429535
レスポンス: 0.013091635890305042


Topic #  15
画像: 0.1373087465763092
検出: 0.03545317053794861
R

## 学習したモデルを保存する

カレントディレクトリと同じ場所に保存すれば、モデルがなければ参照してくれるようになる？

In [29]:
corpora.MmCorpus.serialize("../lda//cop.mm", corpus)
lda.save("../lda/lda.model")

storing corpus in Matrix Market format to ../lda//cop.mm
saving sparse matrix to ../lda//cop.mm
PROGRESS: saving document #0
PROGRESS: saving document #1000
PROGRESS: saving document #2000
PROGRESS: saving document #3000
PROGRESS: saving document #4000
PROGRESS: saving document #5000
PROGRESS: saving document #6000
PROGRESS: saving document #7000
PROGRESS: saving document #8000
saved 8225x13930 matrix, density=0.585% (669846/114574250)
saving MmCorpus index to ../lda//cop.mm.index
saving LdaState object under ../lda/lda.model.state, separately None
saved ../lda/lda.model.state
saving LdaModel object under ../lda/lda.model, separately ['expElogbeta', 'sstats']
storing np array 'expElogbeta' to ../lda/lda.model.expElogbeta.npy
not storing attribute id2word
not storing attribute state
not storing attribute dispatcher
saved ../lda/lda.model


# トピック数の検討

モデルの評価指標として一般的に用いられる Perplexity と Coherence を用いて、トピック数を検討していこうと思います。
optimize_LDAに移動

## 1.perplexity

Perplexity を数式で書くと下記のようになります。
\begin{eqnarray}
  PPL=\exp({-\frac{1}{N}\sum_{i=1}^{N}log_{2}p(w_{i}|θ)})
\end{eqnarray}

N：出現単語数  
p(wi|θ)：周辺の単語に対するある単語 wiの発生確率

特徴しては、負の対数尤度であり、また、一般的にトピック数を増やすと共に低くなる傾向にあり、低い程良いとされます。

In [30]:
corpus = [d.doc2bow(w) for w in share_list]
test_size = int(len(corpus) * 0.1)
test_corpus = corpus[:test_size]
train_corpus= corpus[test_size:]
print("test size: ", len(test_corpus))
print("train size: ", len(train_corpus))

test size:  822
train size:  7403


In [31]:
N = sum(count for doc in test_corpus for id, count in doc)
print("N: ",N)

N:  119601


In [32]:
print("perplexity:", np.exp2(-lda.log_perplexity(test_corpus)))

-9.473 per-word bound, 710.7 perplexity estimate based on a held-out corpus of 822 documents with 119601 words


perplexity: 710.7104082966849


## 2.coherence
Coherence はトピックの品質を測る指標であり、抽出されたトピックが人間にとって解釈しやすいかどうかを表します。

計算方法としては、学習コーパスより、トピックごとの単語間類似度の平均を対数条件付き確率により求めていく流れであり、トピック全体の Coherence が高ければ、良いモデルと見做します。

In [0]:
from collections import defaultdict

# setting frequency
frequency = defaultdict(int)

# count the number of occurrences of the word
for text in share_list:
    for token in text:
        frequency[token] += 1

# build only words above 100 into an array
texts = [[token for token in text if frequency[token] > 100] for text in share_list]

In [34]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, texts=share_list, dictionary=d, coherence='c_v')
print(coherence_model_lda.get_coherence())

using WordOccurrenceAccumulator to estimate probabilities from sliding windows
WordOccurrenceAccumulator accumulated stats from 1000 documents
WordOccurrenceAccumulator accumulated stats from 2000 documents
WordOccurrenceAccumulator accumulated stats from 3000 documents
WordOccurrenceAccumulator accumulated stats from 4000 documents
WordOccurrenceAccumulator accumulated stats from 5000 documents
WordOccurrenceAccumulator accumulated stats from 6000 documents
WordOccurrenceAccumulator accumulated stats from 7000 documents
WordOccurrenceAccumulator accumulated stats from 8000 documents
WordOccurrenceAccumulator accumulated stats from 9000 documents
WordOccurrenceAccumulator accumulated stats from 10000 documents
WordOccurrenceAccumulator accumulated stats from 11000 documents
WordOccurrenceAccumulator accumulated stats from 12000 documents
WordOccurrenceAccumulator accumulated stats from 13000 documents
WordOccurrenceAccumulator accumulated stats from 14000 documents
WordOccurrenceAccumu

0.48417609848165827


Perplexity は低ければ低い程，Coherence は高ければ高い程、良いモデルであると考えられます。